In [2]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [60]:
# htemp=KP(s0,sx,sx,s0)+KP(s0,sy,sy,s0)+KP(s0,sz,sz,s0)
# htemp=KP(sx,sx,s0,s0)+KP(sy,sy,s0,s0)+KP(sz,sz,s0,s0)
# htemp=KP(sz,sz)+(KP(sx,s0)+KP(s0,sx))/4

# d=int(np.sqrt(htemp.shape[0]))
# Ainit=expm(-htemp).reshape(d,d,d,d)
# A0=Asplit(Ainit,chi)

na=np.newaxis
s=np.array([1,-1])
Tc=2/np.log(1+np.sqrt(2))
T=0.9*Tc
A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
                +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
A0/=np.linalg.norm(A0)

chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,6,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

In [61]:
RGsteps=12
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list=[[None]*RGsteps for i in range(8)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,A,Anorm=doTNR(Aold,chi)
#     if A.shape==Aold.shape:
#         A=fix_gauge(Aold,A,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,A,Anorm
    print()

RG step:  0
iter=0, U.Uenv=0.047192, vl.vlenv=0.048804, vr.vrenv=0.0495053
iter=100, U.Uenv=0.143188, vl.vlenv=0.143188, vr.vrenv=0.143188
iter=200, U.Uenv=0.143194, vl.vlenv=0.143194, vr.vrenv=0.143194
iter=300, U.Uenv=0.143197, vl.vlenv=0.143197, vr.vrenv=0.143197
iter=400, U.Uenv=0.143199, vl.vlenv=0.143199, vr.vrenv=0.143199
iter=500, U.Uenv=0.1432, vl.vlenv=0.1432, vr.vrenv=0.1432
iter=600, U.Uenv=0.143201, vl.vlenv=0.143201, vr.vrenv=0.143201
iter=700, U.Uenv=0.143202, vl.vlenv=0.143202, vr.vrenv=0.143202
iter=800, U.Uenv=0.143202, vl.vlenv=0.143202, vr.vrenv=0.143202
iter=900, U.Uenv=0.143203, vl.vlenv=0.143203, vr.vrenv=0.143203
iter=1000, U.Uenv=0.143203, vl.vlenv=0.143203, vr.vrenv=0.143203
error in U, vl, vr optimization: 0.000505882
error in yl, yr, D optimization: 0.307974
error in w optimization: 0.253795

RG step:  1
iter=0, U.Uenv=5.9777e-05, vl.vlenv=7.94111e-05, vr.vrenv=0.000112608
iter=100, U.Uenv=0.0202011, vl.vlenv=0.0202011, vr.vrenv=0.0202011
iter=200, U.Uenv=0.

In [173]:
# def Az_(Ainit,chi):
#     #input: Ainit, chi, op: operator acting on a leg
#     #output:up-down symmetric Aout
#     chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']
#     As=Ainit.shape
    
#     #vertical cut
#     Av=Ainit.reshape(As[0]*As[1],As[2]*As[3])
#     dtemp,vtemp=eigCut(Av,chiw)
#     v=(vtemp*np.sqrt(dtemp)).reshape(As[0],As[1],min(chiw,As[0]*As[1]))

#     #horizontal cut
#     vl2,w,vr2= h_cut(Ainit,chiy)
#     Aout=np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],v.conj(),[4,6,3])
#     Azout=0.5*(np.einsum(vr2,[4,5,0],v,[5,7,1],sz@vl2*w,[6,7,2],v.conj(),[4,6,3])+np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],sz@v.conj(),[4,6,3]))
#     Anorm=np.linalg.norm(Aout)
#     return Azout/Anorm

# A0=Asplit(Ainit,chi)
# Az=Az_(Ainit,chi)
# def MZ_(A0,Az):
#     return np.einsum(Az,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10])

In [62]:
def MA_(A0):
    return np.einsum(A0,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10])

def MZ_(A0):
    return np.einsum(A0,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0*s,[11,7,5,10])
def Mtrace_(MA):
    return np.einsum(MA,[0,1,2,2,3,3,0,1])
MA_list=[None]*(RGsteps+1)
MZ_list=[None]*(RGsteps+1)
MA=MA_(A0)
MAtrace=Mtrace_(MA)
MA_list[0]=MA/MAtrace
MZ_list[0]=MZ_(A0)/MAtrace
print('1=',Mtrace_(MA_list[0]))
print('mag=',Mtrace_(MZ_list[0]))

1= 1.0
mag= 1.1015494072452725e-16


In [63]:
for i in range(RGsteps):
    MAold=MA_list[i]
    MZold=MZ_list[i]
    A,U,vl,vr,yl,yr,D,w=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i]
    MA=Mscaled_(MAold,A,U,vl,vr,yl,yr,D,w)
    MZ=Mscaled_(MZold,A,U,vl,vr,yl,yr,D,w)
    MAtrace=Mtrace_(MA)
    MA_list[i+1]=MA/MAtrace
    MZ_list[i+1]=MZ/MAtrace
    print('mag=',chop(Mtrace_(MZ/MAtrace)))
    #print('1=',Mtrace_(MA/MAtrace))

mag= (0.018306522909348245+0j)
mag= (0.25549254550887984+0j)
mag= (0.6805666966988693+0j)
mag= (0.8433469657846033+0j)
mag= (0.8946400134171183+0j)
mag= (0.8946406747133976+0j)
mag= (0.8946406770877225+0j)
mag= (0.8946406770850769+0j)
mag= (0.8946406770863412+0j)
mag= (0.8946406770873997+0j)
mag= (0.8946406770877517+0j)
mag= (0.8946406770895379+0j)


In [64]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.8959375370799653
